<a href="https://colab.research.google.com/github/NATO-dotcom/ML-AI_Python/blob/main/Smart_pace_trial_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import numpy as np
df=pd.read_csv('/content/study_recommendation_dataset_refined.csv')
df

,subject_type,time_of_day,user_focus_score,task_difficulty,prev_method_used,recommended_duration,recommended_method,mood,user_type
0,coding,evening,7,3,group_discussion,104,project_work,neutral,intermediate
1,coding,evening,1,3,quizzes,39,coding,neutral,advanced
2,theory,afternoon,10,3,quizzes,57,reading,neutral,beginner
3,coding,evening,8,1,quizzes,59,coding,tired,advanced
4,theory,morning,3,2,quizzes,24,reading,neutral,intermediate
...,...,...,...,...,...,...,...,...,...
495,coding,evening,10,1,solo,120,coding,motivated,advanced
496,theory,morning,5,2,group_discussion,44,group_discussion,neutral,beginner
497,theory,morning,8,1,quizzes,82,reading,motivated,advanced
498,math,morning,6,2,solo,62,problem_solving,neutral,advanced


In [11]:
from sklearn.model_selection import train_test_split
from sklearn .preprocessing import LabelEncoder
categorical_columns = ['subject_type', 'time_of_day', 'prev_method_used', 'recommended_method', 'mood', 'user_type']
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le  # Save encoder for decoding later
#features(inputs)
X = df.drop(['recommended_duration','recommended_method'],axis=1)
#targets(outputs)
y_duration = df['recommended_duration']
y_method = df['recommended_method']
#spit into training/testing
X_train, X_test, y_dur_train, y_dur_test, y_method_train, y_method_test = train_test_split(X, y_duration, y_method, test_size=0.2, random_state=42)

In [12]:
from sklearn.ensemble import GradientBoostingClassifier,GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, r2_score, accuracy_score, classification_report
#training Gboost Regressor(for duration)
gboost_regressor = GradientBoostingRegressor(n_estimators=100,learning_rate=0.1,max_depth=3, random_state=42)
gboost_regressor.fit(X_train, y_dur_train)
#predict and evaluate
pred_duration = gboost_regressor.predict(X_test)
print("MAE_duration:",mean_absolute_error(y_dur_test, pred_duration))
print("r2_score:",r2_score(y_dur_test, pred_duration))

#training Gboost Classifier(for method)
gboost_classifier = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gboost_classifier.fit(X_train, y_method_train)
#predict and evaluate
pred_method = gboost_classifier.predict(X_test)
print("accuracy:",accuracy_score(y_method_test, pred_method))
print("classification_report:",classification_report(y_method_test, pred_method))


MAE_duration: 9.894964640973006
r2_score: 0.8152465645182614
accuracy: 0.55
classification_report:               precision    recall  f1-score   support

           0       0.57      0.44      0.50        18
           1       0.58      0.58      0.58        19
           2       0.50      0.46      0.48        13
           3       0.52      0.65      0.58        17
           4       0.59      0.62      0.61        16
           5       0.53      0.53      0.53        17

    accuracy                           0.55       100
   macro avg       0.55      0.55      0.55       100
weighted avg       0.55      0.55      0.55       100



In [13]:
def recommend(subject_type, time_of_day, focus_score, task_difficulty, prev_method, mood, user_type):
    try:
        input_data = pd.DataFrame([[
            label_encoders['subject_type'].transform([subject_type])[0],
            label_encoders['time_of_day'].transform([time_of_day])[0],
            focus_score,
            task_difficulty,
            label_encoders['prev_method_used'].transform([prev_method])[0],
            label_encoders['mood'].transform([mood])[0],
            label_encoders['user_type'].transform([user_type])[0],
        ]], columns=X.columns)
    except Exception as e:
        return {"error": f"Input encoding error: {str(e)}"}

    duration_prediction = int(gboost_regressor.predict(input_data)[0])
    method_encoded = gboost_classifier.predict(input_data)[0]
    method_prediction = label_encoders['recommended_method'].inverse_transform([method_encoded])[0]

    return {
        'recommended_duration': duration_prediction,
        'recommended_method': method_prediction
    }

print(recommend('math', 'afternoon', 7, 2, 'solo', 'motivated', 'intermediate'))

{'recommended_duration': 68, 'recommended_method': 'quizzes'}
